In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path

Part 1. Auxiliary functions

In [ ]:
def read_attributes(path_data) -> pd.DataFrame:
    """Read the catchments` attributes
    Parameters
    ----------
    path_data : str
        patha data

    Returns
    -------
    df: pd.DataFrame
        Dataframe with the catchments` attributes
    """
    # files that contain the attributes
    path_attributes = Path(path_data)
    read_files = list(path_attributes.glob('*_attributes.csv'))

    dfs = []
    # Read each CSV file into a DataFrame and store it in list
    for file in read_files:
        df = pd.read_csv(file, sep=',', header=0, dtype={'gauge_id': str})
        df.set_index('gauge_id', inplace=True)
        dfs.append(df)
    
    # Join all dataframes
    df_attributes= pd.concat(dfs, axis=1)

    return df_attributes


def read_data(path_data, catch_id: str)-> pd.DataFrame:
    """Read the catchments` timeseries

    Parameters
    ----------
    catch_id : str
        identifier of the basin.

    Returns
    -------
    df: pd.DataFrame
        Dataframe with the catchments` timeseries
    """
    path_timeseries = Path(path_data) / 'timeseries' / f'CAMELS_DE_hydromet_timeseries_{catch_id}.csv'
    # load time series
    df = pd.read_csv(path_timeseries)
    df = df.set_index('date')
    df.index = pd.to_datetime(df.index, format="%Y-%m-%d")
    return df

def hargreaves_adapted(df, lat):
    """
    Adapted formulation of the Hargreaves equation to calculate potential evapotranspiration (PET).
    The calculation includes corrections for orographic effects and considers solar geometry based on latitude.

    Parameters:
        df (pandas.DataFrame): DataFrame containing daily weather data with columns for 'temperature_max',
                               'temperature_min', 'precipitation_mean', and 'temperature_mean'. Index should be a datetime type.
        lat (float): Latitude in degrees.

    Returns:
        pandas.Series: Potential evapotranspiration for each day in mm/day.
    """

    t_diff = (df['temperature_max'] - df['temperature_min']).resample('M').transform('mean').reindex(df.index, method='ffill')
    mon_prec = df['precipitation_mean'].resample('M').transform('mean').reindex(df.index, method='ffill')

    # Constants and calculations for solar geometry
    phi = np.radians(lat)  # Convert latitude to radians
    delta = 0.4093 * np.sin(np.radians((360 / 365) * (df.index.day_of_year.values - 39)))  # Solar declination
    dr = 1 + 0.033 * np.cos(np.radians((360 / 365) * df.index.day_of_year.values))  # Earth-sun distance factor
    omega_s = np.arccos(-np.tan(phi) * np.tan(delta))  # Sunset hour angle

    # Handling of polar day/night conditions
    omega_s[np.tan(phi) * np.tan(delta) < -1] = 0  # Sun never rises
    omega_s[np.tan(phi) * np.tan(delta) > 1] = np.pi  # Sun never sets

    # Extraterrestrial radiation for a horizontal surface
    S0 = 15.392 * dr * (omega_s * np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.sin(omega_s))

    # Potential evapotranspiration calculation
    pet = 0.0013 * S0 * (df['temperature_mean'] + 17) * (t_diff - 0.0123 * mon_prec) ** 0.76
    pet[pet < 0] = 0  # Ensuring PET is not negative

    return pet

def penman_pet(df, albedo=0.23):
    # Constants
    cp = 1.013E-3  # Specific heat of air at constant pressure (MJ/kg/°C)
    rho = 1.225  # Air density (kg/m³)
    lambda_v = 2.45  # Latent heat of vaporization (MJ/kg)
    gamma = 0.066  # Psychrometric constant (kPa/°C)
    
    # Convert radiation from W/m² to MJ/m²/day
    radiation_mj = df['radiation_global_mean'].values * 0.0864
    # Calculate net radiation (Rn), assuming no soil heat flux during the day
    Rn = (1 - albedo) * radiation_mj
    # Saturation vapor pressure (es) and actual vapor pressure (ea)
    es = 0.6108 * np.exp((17.27 * df['temperature_mean']) / (df['temperature_mean'] + 237.3))
    ea = es * (df['humidity_mean'] / 100)
    # Slope of the saturation vapor pressure curve (Delta)
    Delta = 4098 * es / ((df['temperature_mean'] + 237.3)**2)
    # Assume typical values for aerodynamic resistance (ra) without wind data
    ra = 208 / np.sqrt(2)  # Typical value for moderate conditions
    # Penman equation
    numerator = (Delta * Rn) + (rho * cp * (es - ea) / ra)
    denominator = lambda_v * (Delta + gamma * (1 + ra / 208))
    PET = numerator / denominator
    
    return PET

Part 2. Input data

In [ ]:
# Path to input data
path_entities = "../../data/basin_id/basins_camels_de_1583.txt"
path_data = "../../data/CAMELS_DE"

Part 3. Calculate PET using hargreaves

In [ ]:
# Read basins
entities_ids = np.loadtxt(path_entities, dtype="str").tolist()

# Calculate evapotranspiration
pet_hargreaves = {}
df_attributes = read_attributes(path_data = path_data)
for id in entities_ids:
    df_data = read_data(path_data = path_data, catch_id=id)
    
    pet= hargreaves_adapted(df = df_data,
                            lat= df_attributes.loc[id, "gauge_lat"], 
)
    
    pet[pet<0.0] = 0.0 # Limit values to zero
    pet_hargreaves[id] = pd.DataFrame(pet, columns=["pet(mm/day)"], index=df_data.index)

with open("pet_hargreaves.pickle", "wb") as f:
    pickle.dump(pet_hargreaves, f)

Part 4. Calculate PET using penman

In [ ]:
# Read basins
entities_ids = np.loadtxt(path_entities, dtype="str").tolist()

# Calculate evapotranspiration
pet_penman = {}
df_attributes = read_attributes(path_data = path_data)
for id in entities_ids:
    df_data = read_data(path_data = path_data, catch_id=id)
    
    pet= penman_pet(df = df_data)
    
    pet[pet<0.0] = 0.0 # Limit values to zero
    pet_penman[id] = pd.DataFrame(pet, columns=["pet(mm/day)"], index=df_data.index)

with open("pet_penman.pickle", "wb") as f:
    pickle.dump(pet_penman, f)

Part 5. Comparte results

In [ ]:
basin_id = "DE112120" # DE110040

plt.rcParams["figure.figsize"] = (20, 10)
plt.plot(pet_hargreaves[basin_id][:1400], label = 'hargreaves')
plt.plot(pet_penman[basin_id][:1400], label = 'penman')
plt.legend()